## Problem 2.2 (Exploring fish sleep data)

In [1]:
import numpy as np
import pandas as pd
import itertools

import altair as alt
import bokeh.io

import bebi103
import altair_catplot as altcat

bokeh.io.output_notebook()

Loading BokehJS ...

In [4]:
df_gt = pd.read_csv('../data/130315_1A_genotypes.txt',
                    delimiter='\t',
                    comment='#',
                    header=[0, 1])
# Take a look
df_gt.head()

,Genotype1,Genotype2,Genotype3
,WT 17,Het 34,Mut 22
0,2.0,1,4.0
1,14.0,3,11.0
2,18.0,5,12.0
3,24.0,6,13.0
4,28.0,8,20.0


In [5]:
# Reset the columns to be the second level of indexing
df_gt.columns = df_gt.columns.get_level_values(1)

# Check out the new columns
df_gt.columns

Index(['WT 17', 'Het 34', 'Mut 22'], dtype='object')

In [6]:
df_gt.columns = ['wt', 'het', 'mut']


In [7]:
# Tidy the DataFrame
df_gt = pd.melt(df_gt, var_name='genotype', value_name='location')

# Take a look
df_gt

,genotype,location
0,wt,2.0
1,wt,14.0
2,wt,18.0
3,wt,24.0
4,wt,28.0
5,wt,29.0
6,wt,30.0
7,wt,54.0
8,wt,58.0
9,wt,61.0


In [8]:
# Drop all rows that have a NaN in them
df_gt = df_gt.dropna()

# Take a look
df_gt

,genotype,location
0,wt,2.0
1,wt,14.0
2,wt,18.0
3,wt,24.0
4,wt,28.0
5,wt,29.0
6,wt,30.0
7,wt,54.0
8,wt,58.0
9,wt,61.0


In [9]:
df_gt = df_gt.reset_index(drop=True)


In [10]:
df_gt.loc[:,'location'] = df_gt.loc[:, 'location'].astype(int)


In [11]:
df = pd.read_csv('../data/130315_1A_aanat2.csv', comment='#')

# Take a look
df.head()

,location,activity,time,zeit,zeit_ind,day
0,1,0.6,2013-03-15 18:31:09,-14.480833,-869,4
1,2,1.4,2013-03-15 18:31:09,-14.480833,-869,4
2,3,0.0,2013-03-15 18:31:09,-14.480833,-869,4
3,4,0.0,2013-03-15 18:31:09,-14.480833,-869,4
4,5,0.0,2013-03-15 18:31:09,-14.480833,-869,4


In [12]:
df = pd.merge(df, df_gt)

# Take a look
df.head()

,location,activity,time,zeit,zeit_ind,day,genotype
0,1,0.6,2013-03-15 18:31:09,-14.480833,-869,4,het
1,1,1.9,2013-03-15 18:32:09,-14.464167,-868,4,het
2,1,1.9,2013-03-15 18:33:09,-14.447500,-867,4,het
3,1,13.4,2013-03-15 18:34:09,-14.430833,-866,4,het
4,1,15.4,2013-03-15 18:35:09,-14.414167,-865,4,het


In [13]:
df['time'] = pd.to_datetime(df['time'])

In [14]:
df['time'].dt.time.head()

0    18:31:09
1    18:32:09
2    18:33:09
3    18:34:09
4    18:35:09
Name: time, dtype: object

In [15]:
df['light'] = (  (df['time'].dt.time >= pd.to_datetime('9:00:00').time())
               & (df['time'].dt.time < pd.to_datetime('23:00:00').time()))

# Take a look
df.head()

,location,activity,time,zeit,zeit_ind,day,genotype,light
0,1,0.6,2013-03-15 18:31:09,-14.480833,-869,4,het,True
1,1,1.9,2013-03-15 18:32:09,-14.464167,-868,4,het,True
2,1,1.9,2013-03-15 18:33:09,-14.447500,-867,4,het,True
3,1,13.4,2013-03-15 18:34:09,-14.430833,-866,4,het,True
4,1,15.4,2013-03-15 18:35:09,-14.414167,-865,4,het,True


In [23]:
df.loc[df['day'] == 5, :]

,location,activity,time,zeit,zeit_ind,day,genotype,light
869,1,0.0,2013-03-16 09:00:09,0.002500,0,5,het,True
870,1,0.0,2013-03-16 09:01:09,0.019167,1,5,het,True
871,1,0.0,2013-03-16 09:02:09,0.035833,2,5,het,True
872,1,0.0,2013-03-16 09:03:09,0.052500,3,5,het,True
873,1,4.2,2013-03-16 09:04:09,0.069167,4,5,het,True
874,1,4.5,2013-03-16 09:05:09,0.085833,5,5,het,True
875,1,6.4,2013-03-16 09:06:09,0.102500,6,5,het,True
876,1,8.6,2013-03-16 09:07:09,0.119167,7,5,het,True
877,1,8.4,2013-03-16 09:08:09,0.135833,8,5,het,True
878,1,8.9,2013-03-16 09:09:09,0.152500,9,5,het,True


In [16]:
grouped = df.groupby('location')


In [17]:
df_median = grouped.median()

df_median.head()

,activity,zeit,zeit_ind,day,light
location,,,,,
1,2.1,30.261111,1816,6,True
2,1.4,30.261111,1816,6,True
3,0.3,30.261111,1816,6,True
4,3.0,30.261111,1816,6,True
5,2.8,30.261111,1816,6,True


In [18]:
df_median = df_median.reset_index()

In [20]:
df_median

,location,activity,zeit,zeit_ind,day,light
0,1,2.1,30.261111,1816,6,True
1,2,1.4,30.261111,1816,6,True
2,3,0.3,30.261111,1816,6,True
3,4,3.0,30.261111,1816,6,True
4,5,2.8,30.261111,1816,6,True
5,6,1.3,30.261111,1816,6,True
6,8,0.5,30.261111,1816,6,True
7,10,0.4,30.261111,1816,6,True
8,11,2.5,30.261111,1816,6,True
9,12,2.3,30.261111,1816,6,True
